In [1]:
import pandas as pd
import numpy as np
import io
import os
import requests
import json
import csv
from io import StringIO
import boto3
from datetime import datetime, timedelta

## Historical data - reading in files from FTP and filtering for US data only

In [ ]:
# Importing the csv file
column_names = ['station_id', 'date', 'metric','values','B','C','D','E']

year_2023_df = pd.read_csv("2023.csv", header=None,names=column_names)

In [3]:
year_2023_df.head(100)

,station_id,date,metric,values,B,C,D,E
0,AE000041196,20230101,TMAX,252,NaN,NaN,S,NaN
1,AE000041196,20230101,TMIN,149,NaN,NaN,S,NaN
2,AE000041196,20230101,PRCP,0,D,NaN,S,NaN
3,AE000041196,20230101,TAVG,207,H,NaN,S,NaN
4,AEM00041194,20230101,TMAX,255,NaN,NaN,S,NaN
...,...,...,...,...,...,...,...,...
95,AGM00060507,20230101,TAVG,143,H,NaN,S,NaN
96,AGM00060511,20230101,TMAX,170,NaN,NaN,S,NaN
97,AGM00060511,20230101,PRCP,0,NaN,NaN,S,NaN
98,AGM00060511,20230101,TAVG,67,H,NaN,S,NaN


In [4]:
# Filtering only the climate metrics we need
year_2023_filter = year_2023_df.loc[year_2023_df['metric'].isin(['TMAX','TMIN','SNOW','PRCP','TAVG'])]

In [5]:
# Grouping and pivoting the dataframe to transpose data
grouped_2023_df_pivot = year_2023_filter.pivot(index=['station_id', 'date'], columns='metric', values='values')

In [6]:
grouped_2023_df_pivot.reset_index(inplace=True) # Resets the index, makes metric a column

In [7]:
grouped_2023_df_pivot.head()

metric,station_id,date,PRCP,SNOW,TAVG,TMAX,TMIN
0,AE000041196,20230101,0.0,NaN,207.0,252.0,149.0
1,AE000041196,20230102,0.0,NaN,213.0,NaN,156.0
2,AE000041196,20230103,0.0,NaN,213.0,264.0,NaN
3,AE000041196,20230104,0.0,NaN,223.0,NaN,168.0
4,AE000041196,20230105,0.0,NaN,229.0,281.0,177.0


In [8]:
grouped_2023_df_pivot.dtypes

metric
station_id     object
date            int64
PRCP          float64
SNOW          float64
TAVG          float64
TMAX          float64
TMIN          float64
dtype: object

In [9]:
# Extracting country code from station ID
grouped_2023_df_pivot['country_code'] = grouped_2023_df_pivot['station_id'].str[:2]

In [10]:
# Further filtering for US only
grouped_2023_df_pivot_US = grouped_2023_df_pivot.loc[grouped_2023_df_pivot["country_code"] == 'US'] 

In [11]:
# Random checks
grouped_2023_df_pivot_US.loc[grouped_2023_df_pivot_US["station_id"] == 'US1CAAL0001']

metric,station_id,date,PRCP,SNOW,TAVG,TMAX,TMIN,country_code
3340384,US1CAAL0001,20230101,696.0,NaN,NaN,NaN,NaN,US
3340385,US1CAAL0001,20230102,0.0,0.0,NaN,NaN,NaN,US
3340386,US1CAAL0001,20230103,112.0,NaN,NaN,NaN,NaN,US
3340387,US1CAAL0001,20230104,64.0,NaN,NaN,NaN,NaN,US
3340388,US1CAAL0001,20230105,455.0,NaN,NaN,NaN,NaN,US
...,...,...,...,...,...,...,...,...
3340692,US1CAAL0001,20231122,0.0,0.0,NaN,NaN,NaN,US
3340693,US1CAAL0001,20231123,0.0,0.0,NaN,NaN,NaN,US
3340694,US1CAAL0001,20231124,0.0,0.0,NaN,NaN,NaN,US
3340695,US1CAAL0001,20231125,0.0,0.0,NaN,NaN,NaN,US


In [12]:
# saving the dataframe
grouped_2023_df_pivot_US.to_csv('2023_US_NOOA_WeatherData.csv')

In [19]:
# Deleting the dataframe
year_2023_df.drop(year_2023_df.index, inplace=True)

In [ ]:
year_2023_df

In [ ]:
import gc

gc.collect()

del year_2023_df
del grouped_2023_df_pivot_US
del grouped_2023_df_pivot

## 2022 Historical data

In [26]:
# Importing the csv file
column_names = ['station_id', 'date', 'metric','values','B','C','D','E']

year_2022_df = pd.read_csv("2022.csv", header=None,names=column_names)

In [27]:
year_2022_df.head(100)

,station_id,date,metric,values,B,C,D,E
0,AE000041196,20220101,TAVG,204,H,NaN,S,NaN
1,AEM00041194,20220101,TAVG,211,H,NaN,S,NaN
2,AEM00041217,20220101,TAVG,209,H,NaN,S,NaN
3,AEM00041218,20220101,TAVG,207,H,NaN,S,NaN
4,AG000060390,20220101,TAVG,121,H,NaN,S,NaN
...,...,...,...,...,...,...,...,...
95,AGM00060540,20220101,TAVG,102,H,NaN,S,NaN
96,AGM00060549,20220101,TMIN,7,NaN,NaN,S,NaN
97,AGM00060549,20220101,TAVG,90,H,NaN,S,NaN
98,AGM00060550,20220101,PRCP,0,NaN,NaN,S,NaN


In [31]:
# Filtering only the climate metrics we need
year_2022_filter = year_2022_df.loc[year_2022_df['metric'].isin(['TMAX','TMIN','SNOW','PRCP','TAVG'])]

In [32]:
# Grouping and pivoting the dataframe to transpose data
grouped_2022_df_pivot = year_2022_filter.pivot(index=['station_id', 'date'], columns='metric', values='values')

In [33]:
grouped_2022_df_pivot.reset_index(inplace=True) # Resets the index, makes metric a column

In [34]:
grouped_2022_df_pivot.head()

metric,station_id,date,PRCP,SNOW,TAVG,TMAX,TMIN
0,AE000041196,20220101,NaN,NaN,204.0,NaN,NaN
1,AE000041196,20220102,NaN,NaN,226.0,NaN,NaN
2,AE000041196,20220103,NaN,NaN,234.0,NaN,NaN
3,AE000041196,20220104,NaN,NaN,217.0,NaN,NaN
4,AE000041196,20220105,NaN,NaN,209.0,NaN,NaN


In [78]:
grouped_2022_df_pivot.dtypes

metric
PRCP    float64
SNOW    float64
TAVG    float64
TMAX    float64
TMIN    float64
dtype: object

In [35]:
# Extracting country code from station ID
grouped_2022_df_pivot['country_code'] = grouped_2022_df_pivot['station_id'].str[:2]

In [36]:
# Further filtering for US only
grouped_2022_df_pivot_US = grouped_2022_df_pivot.loc[grouped_2022_df_pivot["country_code"] == 'US'] 

In [37]:
# saving the dataframe
grouped_2022_df_pivot_US.to_csv('2022_US_NOOA_WeatherData.csv')

In [38]:
gc.collect()

del year_2022_df
del grouped_2022_df_pivot_US
del grouped_2022_df_pivot

## 2021 Historical dataset

In [40]:
# Importing the csv file
column_names = ['station_id', 'date', 'metric','values','B','C','D','E']

year_2021_df = pd.read_csv("2021.csv", header=None,names=column_names)

In [41]:
# Filtering only the climate metrics we need
year_2021_filter = year_2021_df.loc[year_2021_df['metric'].isin(['TMAX','TMIN','SNOW','PRCP','TAVG'])]

In [42]:
# Grouping and pivoting the dataframe to transpose data
grouped_2021_df_pivot = year_2021_filter.pivot(index=['station_id', 'date'], columns='metric', values='values')

In [43]:
grouped_2021_df_pivot.reset_index(inplace=True) # Resets the index, makes metric a column

In [44]:
grouped_2021_df_pivot.head()

metric,station_id,date,PRCP,SNOW,TAVG,TMAX,TMIN
0,AE000041196,20210101,0.0,NaN,214.0,278.0,NaN
1,AE000041196,20210102,0.0,NaN,211.0,NaN,NaN
2,AE000041196,20210103,0.0,NaN,202.0,273.0,NaN
3,AE000041196,20210104,0.0,NaN,194.0,273.0,120.0
4,AE000041196,20210105,0.0,NaN,186.0,272.0,97.0


In [45]:
# Extracting country code from station ID
grouped_2021_df_pivot['country_code'] = grouped_2021_df_pivot['station_id'].str[:2]

In [46]:
# Further filtering for US only
grouped_2021_df_pivot_US = grouped_2021_df_pivot.loc[grouped_2021_df_pivot["country_code"] == 'US'] 

In [47]:
# saving the dataframe
grouped_2021_df_pivot_US.to_csv('2021_US_NOOA_WeatherData.csv')

In [51]:
gc.collect()

del year_2021_df
del grouped_2021_df_pivot_US
del grouped_2021_df_pivot

## 2020 Historical dataset

In [48]:
# Importing the csv file
column_names = ['station_id', 'date', 'metric','values','B','C','D','E']

year_2020_df = pd.read_csv("2020.csv", header=None,names=column_names)

In [49]:
# Filtering only the climate metrics we need
year_2020_filter = year_2020_df.loc[year_2020_df['metric'].isin(['TMAX','TMIN','SNOW','PRCP','TAVG'])]

In [50]:
# Grouping and pivoting the dataframe to transpose data
grouped_2020_df_pivot = year_2020_filter.pivot(index=['station_id', 'date'], columns='metric', values='values')

In [52]:
grouped_2020_df_pivot.reset_index(inplace=True) # Resets the index, makes metric a column

In [53]:
grouped_2020_df_pivot.head()

metric,station_id,date,PRCP,SNOW,TAVG,TMAX,TMIN
0,AE000041196,20200101,0.0,NaN,211.0,NaN,168.0
1,AE000041196,20200102,0.0,NaN,214.0,NaN,NaN
2,AE000041196,20200103,0.0,NaN,212.0,NaN,154.0
3,AE000041196,20200104,0.0,NaN,219.0,NaN,149.0
4,AE000041196,20200105,0.0,NaN,237.0,NaN,165.0


In [54]:
# Extracting country code from station ID
grouped_2020_df_pivot['country_code'] = grouped_2020_df_pivot['station_id'].str[:2]

In [55]:
# Further filtering for US only
grouped_2020_df_pivot_US = grouped_2020_df_pivot.loc[grouped_2020_df_pivot["country_code"] == 'US'] 

In [56]:
# saving the dataframe
grouped_2020_df_pivot_US.to_csv('2020_US_NOOA_WeatherData.csv')

In [57]:
gc.collect()

del year_2020_df
del grouped_2020_df_pivot_US
del grouped_2020_df_pivot

## Getting a list of all California stations 

In [ ]:

token = {'token': '<token>'}

# Define API parameters for stations
stations_params = {
    'datasetid': 'GHCND',  # GHCND dataset
    'limit': 1000,          # Maximum number of results per page
}


# Define the URL for GHCND stations with FIPS:06 for California
url = f"https://www.ncei.noaa.gov/cdo-web/api/v2/stations?locationid=FIPS:06"

# Initialize an empty list to store all station data
all_station_data_california = []

# Make the initial API request to get the total number of stations
initial_response = requests.get(url, headers=token, params=stations_params)
initial_json = initial_response.json()

# Check if 'metadata' key is present in the response
if 'metadata' in initial_json and 'resultset' in initial_json['metadata']:
    total_stations = initial_json['metadata']['resultset']['count']

    # Determine the number of requests needed based on the total number of stations
    num_requests = -(-total_stations // stations_params['limit'])

    # Make multiple requests to get all stations
    for offset in range(1, num_requests + 1):
        stations_params['offset'] = offset
        response = requests.get(url, headers=token, params=stations_params)
        json_data = response.json()

        # Check if 'results' key is present in the response
        if 'results' in json_data:
            ghcnd_stations = json_data['results']
            all_station_data_california.extend([(station['id'], station['name']) for station in ghcnd_stations])

# Save all station data to a CSV file
csv_file_path = 'all_ghcnd_stations_california.csv'
with open(csv_file_path, 'w', newline='', encoding='utf-8') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(['Station ID', 'Station Name'])  # Header row
    csv_writer.writerows(all_station_data_california)

print(f"All GHCND stations data for California saved to {csv_file_path}")


## Real time data

In [64]:
token = {'token': '<token>'}

# Date definitions
today_date = datetime.now() 
day7_before_today = (today_date - timedelta(7)).strftime('%Y-%m-%d')

stations_params = {
    'limit': 1000, # Maximum number of results per page
    'offset': 1, # Starting point of the results   
    #'startdate': day_before_yday, 
    #'enddate': day_before_yday,
}

# Initialize an empty list to store all station data
data_california = []

# URL with FIPS:06 (California) for a date 7 days prior
url = f"https://www.ncei.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&locationid=FIPS:06&startdate=
{day7_before_today}&enddate={day7_before_today}"

station_data_response = requests.get(url, headers=token, params=stations_params)
# Capturing the initial JSON from the API response
initial_json = station_data_response.json()

# Check if 'metadata' key is present in the response
if 'metadata' in initial_json and 'resultset' in initial_json['metadata']:
    total_stations = initial_json['metadata']['resultset']['count']

    # Determine the number of requests needed based on the total number of stations
    # Multiple requests might be required since limit is 1000 per request
    num_requests = -(-total_stations // stations_params['limit'])

    # Make multiple requests to get all data points
    for offset in range(1, num_requests + 1):
        stations_params['offset'] = offset
        response = requests.get(url, headers=token, params=stations_params)
        json_data = response.json()
        #california_stations_data = json.loads(response.text)['results']
        
        # Check if 'results' key is present in the response
        if 'results' in json_data:
            california_stations_data = json_data['results']
            data_california.extend([(dt['date'], dt['datatype'], 
                                     dt['station'], dt['value']) for dt in california_stations_data])
            
# Storing in a dataframe
data_califronia_df = pd.DataFrame(data_california)
column_names_california_df = ['date', 'metric', 'station','values']
data_califronia_df.columns = column_names_california_df

# Save all station data to a CSV file
csv_file_path = 'california_real_time.csv'
with open(csv_file_path, 'w', newline='', encoding='utf-8') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(['Date', 'datatype', 'Station ID', 'Value'])  # Header row
    csv_writer.writerows(data_california)

print(f"All real-time data (7 days from date of run) for California saved to {csv_file_path}")


All real-time data (7 days from date of run) for California saved to california_real_time.csv


In [65]:
data_califronia_df

,date,metric,station,values
0,2023-11-24T00:00:00,PRCP,GHCND:US1CAAL0001,0
1,2023-11-24T00:00:00,SNOW,GHCND:US1CAAL0001,0
2,2023-11-24T00:00:00,PRCP,GHCND:US1CAAL0003,0
3,2023-11-24T00:00:00,SNOW,GHCND:US1CAAL0003,0
4,2023-11-24T00:00:00,PRCP,GHCND:US1CAAL0004,0
...,...,...,...,...
2995,2023-11-24T00:00:00,PRCP,GHCND:USC00049298,0
2996,2023-11-24T00:00:00,SNOW,GHCND:USC00049298,0
2997,2023-11-24T00:00:00,SNWD,GHCND:USC00049298,0
2998,2023-11-24T00:00:00,TMAX,GHCND:USC00049298,56


In [55]:
# Testing code
token = {'token': '<token>'}

# Date definitions
today_date = datetime.now() 
day7_before_today = (today_date - timedelta(7)).strftime('%Y-%m-%d')

stations_params = {
    'limit': 1000, # Maximum number of results per page
    'offset': 1, # Starting point of the results   
    #'startdate': day_before_yday, 
    #'enddate': day_before_yday,
}


url = f"https://www.ncei.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&locationid=FIPS:06&startdate={day7_before_today}&enddate={day7_before_today}"
#url = "https://www.ncei.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&locationid=FIPS:06&startdate=2023-11-26&enddate=2023-11-26"
station_data_response = requests.get(url, headers=token, params=stations_params)
california_stations_data = json.loads(station_data_response.text)['results']

california_stations_weather_df = pd.DataFrame(california_stations_data)

# Display the DataFrame
print(california_stations_weather_df)

0      PRCP
1      SNOW
2      PRCP
3      SNOW
4      PRCP
       ... 
464    SNOW
465    SNWD
466    TOBS
467    TOBS
468    TOBS
Name: datatype, Length: 469, dtype: object
